# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd

from sqlalchemy import create_engine
import sqlalchemy as sql

from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger']) 
import pickle

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [ ]:
# engine = create_engine('sqlite:///DBAndreeaA.db')
# df.to_sql('input_disaster', engine, index=False)

In [ ]:
# load data from database
engine = create_engine('sqlite:///DBAndreeaA.db')
df = pd.read_sql_table('input_disaster', engine)  
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
targets = [c for c in df.columns if c not in ['id', 'message', 'original', 'genre']]
Y = df[targets]

In [ ]:
Y.head()

### 2. Write a tokenization function to process your text data

In [ ]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

Multi target classification

This strategy consists of fitting one classifier per target. This is a simple strategy for extending classifiers that do not natively support multi-target classification

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=tokenize)
clf = MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)
pipeline = Pipeline([('vect', vectorizer), ('model', clf)])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['message'], Y) #?stratify=Y

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
%%time
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
%%time
pipeline.score(X_test, y_test)

In [ ]:
# Y.columns

In [ ]:
%%time
y_pred = pipeline.predict(X_test)
pred_pd = pd.DataFrame(y_pred)
pred_pd.columns = Y.columns
pred_pd.head()

In [ ]:
for col in Y.columns:  
    print(col + '\n')
    print(classification_report(pred_pd[col], y_test[col]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# RandomForestClassifier().get_params('metric').keys()

In [ ]:
MultiOutputClassifier(RandomForestClassifier()).get_params('metric').keys()

In [ ]:
%%time

parameters = {
            'vect__ngram_range': ((1, 1), (1, 2)),
            'vect__max_df': (0.5, 0.75, 1.0),
            'vect__max_features': (None, 5000, 10000),
            'model__estimator__max_depth':[5,6,7],
            'model__estimator__n_estimators': [50, 100, 200],
            'model__estimator__min_samples_split': [2, 3, 4],
             }

cv = GridSearchCV(estimator = pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

In [ ]:
cv_params = cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)
pred_pd = pd.DataFrame(y_pred)
pred_pd.columns = Y.columns
pred_pd.head()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
XGBoostClassifier/kNNClassifier

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(pipeline, open('pipeline.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.